# Package Load

In [1]:
import preprocessing
import tensorflow as tf
import numpy as np
import random
import sys
import utils
import pickle

# preprocessing : 이미지 파일로 존재하는 데이터를 numpy와 onehotencoding 라벨로 바꿔주는 모듈
# tensorflow : 텐서플로우 , 일반적으로 tf로 줄여서 사용
# numpy : fundamental package for scientific computing with Python
# pickle : python 의 데이터 형식을 그대로 저장하기 위한 파이썬 기본 패키지


# Image Loading

In [2]:
train_imgs , train_labs , test_imgs , test_labs=preprocessing.get_cifar_data()

def next_batch(imgs , labs , batch_size):
    indices=random.sample(range(len(imgs)) , batch_size)
    batch_xs=imgs[indices]
    batch_ys=labs[indices]
    return batch_xs , batch_ys

# preprocessing 모듈에서 train_imgs , train_labs , test_imgs , test_labs 를 불러 온다.
# fully connected 모델을 위해 32x32x3 크기의 이미지를 3072 으로 reshape 한다.
# next_batch 함수는 batch_size 수 만큼 random 으로 추출하여 train batch 를 생성하는 함수


# Set Placeholder And Variables

In [3]:

height = 32
width = 32
color_ch=3
n_classes = 10
learning_rate=0.001
max_iter=10000
check_point=100
x_ =tf.placeholder(tf.float32, [ None , height , width , color_ch ])
y_ =tf.placeholder( tf.int32 , [ None , n_classes ])

# Placeholder 는 차후에 입력할 값
# Variables 는 weight 로 학습시 변하는 값


# Network Modeling

In [4]:
out_ch=28
w1=tf.get_variable("w1" , [7,7,color_ch , out_ch] , initializer=tf.contrib.layers.xavier_initializer())
b1=tf.Variable(tf.constant(0.1) ,out_ch)
s1=[1,1,1,1]
p1='SAME'
layer1=tf.nn.conv2d(x_ , w1 , s1 , p1 )+b1
layer1=tf.nn.relu(layer1)


out_ch2=64
w2=tf.get_variable("w2" , [5,5,out_ch, out_ch2] , initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.constant(0.1) ,out_ch2)
s2=[1,1,1,1]
layer2=tf.nn.conv2d(layer1, w2 , s2, padding='SAME')+b2
layer2=tf.nn.relu(layer2)


pool_s = [1,2,2,1]
layer3 = tf.nn.max_pool(layer2, ksize=[1, 2, 2, 1], strides=pool_s, padding='SAME')

out_ch3=128
w3=tf.get_variable("w3" , [5,5,out_ch2, out_ch3] , initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.constant(0.1) ,out_ch3)
s3=[1,1,1,1]
layer4=tf.nn.conv2d(layer3, w3 , s3, padding='SAME')+b3
layer4=tf.nn.relu(layer4)

out_ch4=128
w4=tf.get_variable("w4" , [5,5,out_ch3, out_ch4] , initializer=tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.constant(0.1) ,out_ch4)
s4=[1,1,1,1]
layer4=tf.nn.conv2d(layer4, w4 , s4, padding='SAME')+b4
layer4=tf.nn.relu(layer4)

layer5 = tf.nn.max_pool(layer4, ksize=[1, 2, 2, 1], strides=pool_s, padding='SAME')

#fully connected layer = affine layer

end_conv_layer=layer5
flatten_layer=tf.contrib.layers.flatten(end_conv_layer)
length=flatten_layer.get_shape()[1]
fc_w1=tf.get_variable("fc_w1" ,[length,n_classes])
fc_b1=tf.Variable(tf.constant(0.1) , n_classes)
y_conv=tf.matmul(flatten_layer ,fc_w1 )+fc_b1

# 4개의 convolution neural network 와 1개의 fully connected_layer 로 구성
# 2개의 convolution layer 를 거친 후 각 각 max pooling 적용 / max pooling 후에는 activation map 의 가로 세로 크기가 절반이 된다.
# stride : 좌우로 몇 칸 씩 커널을 이동 할 것인지에 대한 값
# padding : convolution 전후로 activation map 의 크기를 조정하기 위한 값 , SAME 을 입력하면 항상 convolution 전후의 크기가 같다.
# weight : [커널의 가로크기, 커널의 세로크기, input activation map 의 채널 크기, output activiation map 의 채널 크기]


# Define Functions

In [5]:
pred=tf.nn.softmax(y_conv)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= pred , labels=y_) , name='cost')
train=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost,name='train')
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1), name='correct_pred')
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32), name='accuracy')

# softmax : 결과값을 각 class 에 해당하는 확률값을로 리턴
# softmax_cross_entropy_with_logits : 실제 정답인 onehot vector 와 예측값 pred 를 차이를 cross_entropy 로 계산
# tf.train.GradientDescentOptimizer : cost 가 최소와 되도록 weight를 조정하는 함수
# accuracy : 실제 값과 예측값의 일치률

# Training

In [6]:
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver()

batch_iteration = 100    
training_epochs = 2000

train_cost_list = []
test_cost_list = []
train_acc_list = []
test_acc_list = []

# Tensorflow 학습을 위한 session 정의
# saver : 학습한 weight 와 graph 를 저장

# sess.run 을 통해 함수를 실행 , feed_dict 는 placeholder 에 들어갈 값


for epoch in range(training_epochs):
    
    train_avg_cost = 0.
    train_avg_acc = 0.
    test_avg_cost = 0.
    test_avg_acc = 0.
    
    for batch in range(batch_iteration) :
        batch_xs , batch_ys = next_batch(train_imgs , train_labs , 60)
        
        sess.run(train, feed_dict={x_:batch_xs , y_ : batch_ys})
        train_avg_cost += sess.run(cost, feed_dict={x_:batch_xs , y_ : batch_ys})
        train_avg_acc += sess.run(accuracy, feed_dict={x_:batch_xs , y_ : batch_ys})
        
    train_avg_cost = train_avg_cost / batch_iteration
    train_avg_acc = train_avg_acc / batch_iteration
    
    test_avg_cost = sess.run(cost, feed_dict={x_: test_imgs, y_: test_labs})
    test_avg_acc = sess.run(accuracy, feed_dict={x_: test_imgs, y_: test_labs})
    
    
    print "##################################"
    print ("Epoch: %03d/%03d train cost: %.4f" % (epoch, training_epochs, train_avg_cost))
    print ("Epoch: %03d/%03d train acc: %.4f" % (epoch, training_epochs, train_avg_acc))
    print ("Epoch: %03d/%03d test cost: %.4f" % (epoch, training_epochs, test_avg_cost))
    print ("Epoch: %03d/%03d test acc: %.4f" % (epoch, training_epochs, test_avg_acc))    
    
    train_cost_list.append(train_avg_cost)
    test_cost_list.append(test_avg_cost)
    train_acc_list.append(train_avg_acc)
    test_acc_list.append(train_avg_acc)
   
    
    
saver.save(sess, "model/cifar_deep_convolution.ckpt")

with open('cost_acc/cifar_deep_convolution_train_cost', 'wb') as fp:
    pickle.dump(train_avg_cost, fp)
with open('cost_acc/cifar_deep_convolution_test_cost', 'wb') as fp:
    pickle.dump(test_avg_cost, fp)
with open('cost_acc/cifar_deep_convolution_train_acc', 'wb') as fp:
    pickle.dump(train_avg_acc, fp)
with open('cost_acc/cifar_deep_convolution_test_acc', 'wb') as fp:
    pickle.dump(test_avg_acc, fp)
    

##################################
Epoch: 000/2000 train cost: 2.3024
Epoch: 000/2000 train acc: 0.1057
Epoch: 000/2000 test cost: 2.3028
Epoch: 000/2000 test acc: 0.0980
##################################
Epoch: 001/2000 train cost: 2.3022
Epoch: 001/2000 train acc: 0.1020
Epoch: 001/2000 test cost: 2.3026
Epoch: 001/2000 test acc: 0.0980
##################################
Epoch: 002/2000 train cost: 2.3022
Epoch: 002/2000 train acc: 0.0958
Epoch: 002/2000 test cost: 2.3024
Epoch: 002/2000 test acc: 0.0985
##################################
Epoch: 003/2000 train cost: 2.3020
Epoch: 003/2000 train acc: 0.1008
Epoch: 003/2000 test cost: 2.3023
Epoch: 003/2000 test acc: 0.0985
##################################
Epoch: 004/2000 train cost: 2.3016
Epoch: 004/2000 train acc: 0.1065
Epoch: 004/2000 test cost: 2.3021
Epoch: 004/2000 test acc: 0.0982
##################################
Epoch: 005/2000 train cost: 2.3020
Epoch: 005/2000 train acc: 0.1005
Epoch: 005/2000 test cost: 2.3020
Epoch: 

##################################
Epoch: 048/2000 train cost: 2.2839
Epoch: 048/2000 train acc: 0.1843
Epoch: 048/2000 test cost: 2.2818
Epoch: 048/2000 test acc: 0.1949
##################################
Epoch: 049/2000 train cost: 2.2797
Epoch: 049/2000 train acc: 0.1988
Epoch: 049/2000 test cost: 2.2793
Epoch: 049/2000 test acc: 0.1933
##################################
Epoch: 050/2000 train cost: 2.2747
Epoch: 050/2000 train acc: 0.2088
Epoch: 050/2000 test cost: 2.2764
Epoch: 050/2000 test acc: 0.1995
##################################
Epoch: 051/2000 train cost: 2.2745
Epoch: 051/2000 train acc: 0.1960
Epoch: 051/2000 test cost: 2.2736
Epoch: 051/2000 test acc: 0.2002
##################################
Epoch: 052/2000 train cost: 2.2695
Epoch: 052/2000 train acc: 0.1973
Epoch: 052/2000 test cost: 2.2703
Epoch: 052/2000 test acc: 0.1973
##################################
Epoch: 053/2000 train cost: 2.2639
Epoch: 053/2000 train acc: 0.2110
Epoch: 053/2000 test cost: 2.2672
Epoch: 

##################################
Epoch: 096/2000 train cost: 2.1539
Epoch: 096/2000 train acc: 0.3178
Epoch: 096/2000 test cost: 2.1704
Epoch: 096/2000 test acc: 0.2871
##################################
Epoch: 097/2000 train cost: 2.1586
Epoch: 097/2000 train acc: 0.3082
Epoch: 097/2000 test cost: 2.1647
Epoch: 097/2000 test acc: 0.2984
##################################
Epoch: 098/2000 train cost: 2.1601
Epoch: 098/2000 train acc: 0.3083
Epoch: 098/2000 test cost: 2.1620
Epoch: 098/2000 test acc: 0.3017
##################################
Epoch: 099/2000 train cost: 2.1556
Epoch: 099/2000 train acc: 0.3113
Epoch: 099/2000 test cost: 2.1616
Epoch: 099/2000 test acc: 0.2985
##################################
Epoch: 100/2000 train cost: 2.1461
Epoch: 100/2000 train acc: 0.3233
Epoch: 100/2000 test cost: 2.1606
Epoch: 100/2000 test acc: 0.2973
##################################
Epoch: 101/2000 train cost: 2.1483
Epoch: 101/2000 train acc: 0.3200
Epoch: 101/2000 test cost: 2.1585
Epoch: 

##################################
Epoch: 144/2000 train cost: 2.0970
Epoch: 144/2000 train acc: 0.3655
Epoch: 144/2000 test cost: 2.1250
Epoch: 144/2000 test acc: 0.3334
##################################
Epoch: 145/2000 train cost: 2.1046
Epoch: 145/2000 train acc: 0.3632
Epoch: 145/2000 test cost: 2.1212
Epoch: 145/2000 test acc: 0.3391
##################################
Epoch: 146/2000 train cost: 2.1053
Epoch: 146/2000 train acc: 0.3588
Epoch: 146/2000 test cost: 2.1194
Epoch: 146/2000 test acc: 0.3386
##################################
Epoch: 147/2000 train cost: 2.1018
Epoch: 147/2000 train acc: 0.3653
Epoch: 147/2000 test cost: 2.1195
Epoch: 147/2000 test acc: 0.3376
##################################
Epoch: 148/2000 train cost: 2.1028
Epoch: 148/2000 train acc: 0.3620
Epoch: 148/2000 test cost: 2.1204
Epoch: 148/2000 test acc: 0.3385
##################################
Epoch: 149/2000 train cost: 2.1090
Epoch: 149/2000 train acc: 0.3552
Epoch: 149/2000 test cost: 2.1210
Epoch: 

##################################
Epoch: 192/2000 train cost: 2.0917
Epoch: 192/2000 train acc: 0.3748
Epoch: 192/2000 test cost: 2.1025
Epoch: 192/2000 test acc: 0.3527
##################################
Epoch: 193/2000 train cost: 2.0904
Epoch: 193/2000 train acc: 0.3743
Epoch: 193/2000 test cost: 2.1021
Epoch: 193/2000 test acc: 0.3551
##################################
Epoch: 194/2000 train cost: 2.0817
Epoch: 194/2000 train acc: 0.3842
Epoch: 194/2000 test cost: 2.1034
Epoch: 194/2000 test acc: 0.3540
##################################
Epoch: 195/2000 train cost: 2.0784
Epoch: 195/2000 train acc: 0.3880
Epoch: 195/2000 test cost: 2.1017
Epoch: 195/2000 test acc: 0.3575
##################################
Epoch: 196/2000 train cost: 2.0801
Epoch: 196/2000 train acc: 0.3855
Epoch: 196/2000 test cost: 2.0996
Epoch: 196/2000 test acc: 0.3589
##################################
Epoch: 197/2000 train cost: 2.0825
Epoch: 197/2000 train acc: 0.3822
Epoch: 197/2000 test cost: 2.0990
Epoch: 

...

##################################
Epoch: 1807/2000 train cost: 1.7811
Epoch: 1807/2000 train acc: 0.6850
Epoch: 1807/2000 test cost: 1.9086
Epoch: 1807/2000 test acc: 0.5501
##################################
Epoch: 1808/2000 train cost: 1.7829
Epoch: 1808/2000 train acc: 0.6842
Epoch: 1808/2000 test cost: 1.9047
Epoch: 1808/2000 test acc: 0.5540
##################################
Epoch: 1809/2000 train cost: 1.7834
Epoch: 1809/2000 train acc: 0.6822
Epoch: 1809/2000 test cost: 1.9088
Epoch: 1809/2000 test acc: 0.5475
##################################
Epoch: 1810/2000 train cost: 1.7823
Epoch: 1810/2000 train acc: 0.6845
Epoch: 1810/2000 test cost: 1.9057
Epoch: 1810/2000 test acc: 0.5527
##################################
Epoch: 1811/2000 train cost: 1.7818
Epoch: 1811/2000 train acc: 0.6842
Epoch: 1811/2000 test cost: 1.9070
Epoch: 1811/2000 test acc: 0.5512
##################################
Epoch: 1812/2000 train cost: 1.7746
Epoch: 1812/2000 train acc: 0.6903
Epoch: 1812/2000 te

##################################
Epoch: 1854/2000 train cost: 1.7778
Epoch: 1854/2000 train acc: 0.6868
Epoch: 1854/2000 test cost: 1.9104
Epoch: 1854/2000 test acc: 0.5468
##################################
Epoch: 1855/2000 train cost: 1.7841
Epoch: 1855/2000 train acc: 0.6808
Epoch: 1855/2000 test cost: 1.9081
Epoch: 1855/2000 test acc: 0.5502
##################################
Epoch: 1856/2000 train cost: 1.7736
Epoch: 1856/2000 train acc: 0.6920
Epoch: 1856/2000 test cost: 1.9064
Epoch: 1856/2000 test acc: 0.5523
##################################
Epoch: 1857/2000 train cost: 1.7790
Epoch: 1857/2000 train acc: 0.6853
Epoch: 1857/2000 test cost: 1.9074
Epoch: 1857/2000 test acc: 0.5499
##################################
Epoch: 1858/2000 train cost: 1.7654
Epoch: 1858/2000 train acc: 0.7002
Epoch: 1858/2000 test cost: 1.9052
Epoch: 1858/2000 test acc: 0.5507
##################################
Epoch: 1859/2000 train cost: 1.7737
Epoch: 1859/2000 train acc: 0.6915
Epoch: 1859/2000 te

##################################
Epoch: 1901/2000 train cost: 1.7624
Epoch: 1901/2000 train acc: 0.7028
Epoch: 1901/2000 test cost: 1.9070
Epoch: 1901/2000 test acc: 0.5515
##################################
Epoch: 1902/2000 train cost: 1.7659
Epoch: 1902/2000 train acc: 0.6997
Epoch: 1902/2000 test cost: 1.9064
Epoch: 1902/2000 test acc: 0.5527
##################################
Epoch: 1903/2000 train cost: 1.7645
Epoch: 1903/2000 train acc: 0.7018
Epoch: 1903/2000 test cost: 1.9068
Epoch: 1903/2000 test acc: 0.5505
##################################
Epoch: 1904/2000 train cost: 1.7582
Epoch: 1904/2000 train acc: 0.7062
Epoch: 1904/2000 test cost: 1.9048
Epoch: 1904/2000 test acc: 0.5512
##################################
Epoch: 1905/2000 train cost: 1.7703
Epoch: 1905/2000 train acc: 0.6955
Epoch: 1905/2000 test cost: 1.9070
Epoch: 1905/2000 test acc: 0.5510
##################################
Epoch: 1906/2000 train cost: 1.7727
Epoch: 1906/2000 train acc: 0.6928
Epoch: 1906/2000 te

##################################
Epoch: 1948/2000 train cost: 1.7637
Epoch: 1948/2000 train acc: 0.7018
Epoch: 1948/2000 test cost: 1.9061
Epoch: 1948/2000 test acc: 0.5533
##################################
Epoch: 1949/2000 train cost: 1.7625
Epoch: 1949/2000 train acc: 0.7015
Epoch: 1949/2000 test cost: 1.9066
Epoch: 1949/2000 test acc: 0.5518
##################################
Epoch: 1950/2000 train cost: 1.7687
Epoch: 1950/2000 train acc: 0.6957
Epoch: 1950/2000 test cost: 1.9036
Epoch: 1950/2000 test acc: 0.5520
##################################
Epoch: 1951/2000 train cost: 1.7675
Epoch: 1951/2000 train acc: 0.6965
Epoch: 1951/2000 test cost: 1.9044
Epoch: 1951/2000 test acc: 0.5517
##################################
Epoch: 1952/2000 train cost: 1.7589
Epoch: 1952/2000 train acc: 0.7050
Epoch: 1952/2000 test cost: 1.9048
Epoch: 1952/2000 test acc: 0.5540
##################################
Epoch: 1953/2000 train cost: 1.7614
Epoch: 1953/2000 train acc: 0.7022
Epoch: 1953/2000 te

##################################
Epoch: 1995/2000 train cost: 1.7745
Epoch: 1995/2000 train acc: 0.6907
Epoch: 1995/2000 test cost: 1.9046
Epoch: 1995/2000 test acc: 0.5533
##################################
Epoch: 1996/2000 train cost: 1.7686
Epoch: 1996/2000 train acc: 0.6960
Epoch: 1996/2000 test cost: 1.9020
Epoch: 1996/2000 test acc: 0.5553
##################################
Epoch: 1997/2000 train cost: 1.7683
Epoch: 1997/2000 train acc: 0.6962
Epoch: 1997/2000 test cost: 1.9051
Epoch: 1997/2000 test acc: 0.5524
##################################
Epoch: 1998/2000 train cost: 1.7745
Epoch: 1998/2000 train acc: 0.6888
Epoch: 1998/2000 test cost: 1.9042
Epoch: 1998/2000 test acc: 0.5521
##################################
Epoch: 1999/2000 train cost: 1.7686
Epoch: 1999/2000 train acc: 0.6965
Epoch: 1999/2000 test cost: 1.9083
Epoch: 1999/2000 test acc: 0.5478
